In [1]:
# import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
%ls /kaggle/input/machinelearing/

ls: cannot access '/kaggle/input/machinelearing/': No such file or directory


In [3]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive//preprocess_real_final.csv", encoding =  'cp949')
df = df.drop(['Unnamed: 0'], axis = 1)

Mounted at /content/drive


In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score

In [6]:
X = df.iloc[:,(df.columns != 'price')]
y = df.price
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(35708, 40) (15304, 40) (35708,) (15304,)


In [7]:
my_model = XGBRegressor(n_estimators = 500)
##Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_x, train_y, verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
model = XGBRegressor(max_depth = 3, subsample = 0.9, gamma = 0.0)
model.fit(train_x, train_y, eval_metric='mae')

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(test_x)
mse = mean_squared_error(test_y, y_pred)
r2 = r2_score(test_y, y_pred)
print("MSE: {}".format(mse))
print("R square",r2)

In [ ]:
reg_cv = GridSearchCV(my_model, {'max_depth': [3,4,6], 'subsample':[i/10.0 for i in range(6,10)], 'gamma':[i/10.0 for i in range(0,5)]}, verbose=1)
reg_cv.fit(train_x,train_y)
reg_cv.best_params_

In [ ]:
from xgboost import plot_importance

plot_importance(model)

In [ ]:
import matplotlib.pyplot as plt

pre = pd.DataFrame(y_pred)
test_y = test_y.reset_index()
result = pd.concat([test_y, pre], axis = 1)
result.columns = ['a', 'real', 'predict']

fig = plt.figure(figsize = (12,4))
chart = fig.add_subplot(1,1,1)
chart.plot(result.real[:800],  color = 'blue', label = 'y')
chart.plot(result.predict[:800],  color = 'red', label = 'predict_y')
chart.set_title('XGBoost Result', size=30)
plt.xlabel('num', size = 20)
plt.ylabel('price', size = 20)
plt.legend(loc = 'best')